In [1]:
import xgboost as xgb
from xgboost import cv
import pandas as pd
import numpy as np
from joblib import dump, load
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, RepeatedKFold
from sklearn.metrics import make_scorer, mean_squared_error
from typing import Tuple

In [2]:
pred_cols = ["Mean_BMI","Median_BMI","Unmet_Need_Rate","Under5_Mortality_Rate","Skilled_Birth_Attendant_Rate","Stunted_Rate"]
feature_names = np.load('../data/feature_names.npy', allow_pickle=True)

In [3]:
train = pd.read_parquet('../data/train.parquet.gzip')
train = train[~train.index.duplicated(keep='first')]
X = train.drop(pred_cols, axis=1)
y = train[pred_cols]

In [4]:
del(train)
import gc
gc.collect()

0

In [5]:
# keep only features in feature_names
X = X[feature_names]
X

,Es_min_max@CAS/IGSNRR/PML/V2&timestamped,basic_demographic_characteristics_min_max@CIESIN/GPWv411/GPW_Basic_Demographic_Characteristics&timestamped,national_identifier_grid@CIESIN/GPWv411/GPW_National_Identifier_Grid,urban-coverfraction_mean@COPERNICUS/Landcover/100m/Proba-V-C3/Global&timestamped,SO2_column_number_density_15km_max_max@COPERNICUS/S5P/NRTI/L3_SO2&timestamped,solar_azimuth_angle_max_max@COPERNICUS/S5P/NRTI/L3_SO2&timestamped,ozone_tropospheric_mixing_ratio_median@COPERNICUS/S5P/OFFL/L3_O3_TCL&timestamped,surface_solar_radiation_downwards_median@ECMWF/ERA5_LAND/MONTHLY&timestamped,surface_thermal_radiation_downwards_max_min@ECMWF/ERA5_LAND/MONTHLY&timestamped,LC_Type3_min_max@MODIS/006/MCD12Q1&timestamped,...,onehotencoder__DHSCC_BU,onehotencoder__DHSCC_CO,onehotencoder__DHSCC_DR,onehotencoder__DHSCC_ET,onehotencoder__DHSCC_HT,onehotencoder__DHSCC_MZ,onehotencoder__DHSCC_TZ,onehotencoder__URBAN_RURA_R,remainder__DHSYEAR,remainder__LATNUM
DHSID,,,,,,,,,,,,,,,,,,,,,
AL200800000001,0.079005,0.533688,8.0,15.942918,0.001929,179.557310,21.800858,14866048.0,32168087.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2008.0,40.822650
AL200800000002,0.090663,0.165650,8.0,1.059678,0.001774,179.592150,21.800858,14591484.0,31264967.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2008.0,40.696846
AL200800000004,0.085710,0.814697,8.0,10.031118,0.001929,179.768220,21.800858,14753848.0,31802635.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2008.0,40.798930
AL200800000010,0.105490,8.942595,8.0,22.716291,0.001986,179.742550,21.800858,14591484.0,31264967.0,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2008.0,40.698520
AL200800000011,0.106758,8.940361,8.0,41.684960,0.001609,179.742550,21.800858,14703080.0,31623683.0,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2008.0,40.717968
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZW201500000395,0.007476,0.000000,716.0,0.000000,0.000404,-18.019940,51.753593,19855312.0,33592193.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2015.0,-17.166506
ZW201500000396,0.002424,0.000000,716.0,80.522360,0.000402,-17.357134,53.740124,20481506.0,32498543.0,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2015.0,-17.915289
ZW201500000397,0.038481,0.000000,716.0,0.298818,0.000336,-16.403381,53.757790,21089554.0,32038087.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2015.0,-18.379501


In [6]:
# def mcrmse(predt: np.ndarray, dtrain: xgb.DMatrix) -> Tuple[str, float]:
#     ''' Mean column-wise root mean square error metric.'''
#     y = dtrain.get_label().reshape(-1, 6)
#     # loop over each column and calculate error
#     errors = []
#     for i in range(predt.shape[1]):
#         errors.append(np.sqrt(mean_squared_error(y[:,i], predt[:,i])))
#     ans = np.mean(errors)
#     return 'MCRMSE', float(ans)

In [7]:
# from xgboost import cv

# params = {'colsample_bytree': 0.8,'learning_rate': 0.3,
#                 'max_depth': 10, 'gamma': 1.5, 'min_child_weight': 5}

# xgb_cv = cv(params=params, dtrain=xgb.DMatrix(X, y), num_boost_round=100, nfold=5, 
#             custom_metric=mcrmse, seed=42, as_pandas=True, 
#             callbacks=[xgb.callback.EvaluationMonitor(show_stdv=False),
#                         xgb.callback.EarlyStopping(10)])

In [8]:
def mcrmse(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    return np.average(rmse)

In [9]:
xgb_model = xgb.XGBRegressor(random_state=42)

params = {'colsample_bytree': [0.8,0.9],
          'learning_rate': [0.03,0.1],
          'max_depth': [4,8,12], 
          'gamma': [0,1.5,2.3], 
          'min_child_weight': [5,7]
          } 

model = GridSearchCV(estimator = xgb_model, param_grid = params, verbose=1, 
                     scoring=make_scorer(mcrmse, greater_is_better=False), cv=5, n_jobs=-1)
model.fit(X,y)

model.best_params_

Fitting 5 folds for each of 72 candidates, totalling 360 fits


{'colsample_bytree': 0.9,
 'gamma': 0,
 'learning_rate': 0.1,
 'max_depth': 4,
 'min_child_weight': 7}

In [10]:
best_model = xgb.XGBRegressor(**model.best_params_, random_state=42)
best_model.fit(X,y)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.9, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=4, max_leaves=None,
             min_child_weight=7, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=42, ...)

In [11]:
test = pd.read_parquet('../data/test_all.parquet.gzip')
test = test[feature_names]

In [12]:
y_pred = best_model.predict(test)
out = pd.DataFrame(y_pred, columns=pred_cols)
out['DHSID'] = test.index
out = out[['DHSID'] + pred_cols]
out.to_csv('../submission/new_test.csv', index=False)

In [13]:
out

,DHSID,Mean_BMI,Median_BMI,Unmet_Need_Rate,Under5_Mortality_Rate,Skilled_Birth_Attendant_Rate,Stunted_Rate
0,AL200800000003,24.810574,24.031719,26.183750,5.010227,77.875732,22.245295
1,AL200800000005,24.830389,23.927700,30.259094,4.892070,74.054070,22.511044
2,AL200800000007,25.035278,24.141083,21.580723,3.565645,76.806328,20.488739
3,AL200800000008,24.841982,24.198687,22.966158,3.723278,77.306519,17.970589
4,AL200800000009,24.841982,24.029774,22.966158,3.610278,77.306519,17.970589
...,...,...,...,...,...,...,...
14995,ZW201500000382,24.036409,23.386400,19.050905,7.135818,75.868576,31.557322
14996,ZW201500000383,23.747318,22.999056,18.991081,5.985414,74.593674,29.252737
14997,ZW201500000386,23.745510,23.014353,23.870831,6.444063,65.680717,28.286310
14998,ZW201500000390,25.611238,24.823668,12.330021,4.915627,96.935837,21.782232
